# Imports

In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
import os
import configparser
config = configparser.ConfigParser()
config.read('/home/jovyan/aws/config.cfg')
os.environ['AWS_KEY_ID'] = config.get("AWS", "KEY")
os.environ['AWS_SECRET'] = config.get("AWS", 'SECRET')

### Check that you have the hadoop aws package for your version of hadoop
````
 !ls /usr/local/spark/jars/hadoop* # to figure out what version of hadoop
````

In [3]:
import pyspark
spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.hadoop.fs.s3a.access.key", os.environ.get('AWS_KEY_ID')) \
        .config("spark.hadoop.fs.s3a.secret.key", os.environ.get('AWS_SECRET')) \
        .getOrCreate()


In [4]:
mycsv = "s3a://dendpaulogieruswest2/sampledata/titanic-data.csv"
df = spark.read.option("header", "true").csv(mycsv)
df.count()

891

In [8]:
df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [9]:
spark.stop()

# Write back the results

In [ ]:
output_path = "s3a://mybucket/output/sampletitanic"
df.write.partitionBy("Pclass").parquet(output_path)

# Check that you can read the results

In [ ]:
df2 = spark.read.option("header", "true").parquet(output_path)
df2.count()